In [1]:
from openai import OpenAI
client = OpenAI(api_key="sk-proj-3KhQdlgMCuoQc6EL2y1ZT3BlbkFJYHG9JgPGpOLsRnDSKuSs")

In [2]:
import json
import pandas as pd
import time

In [4]:
to_summarize = pd.read_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\Summarizing contradictions - Germany.csv")

In [9]:
to_summarize = pd.read_csv(r"summarized_by_gpt.csv")

In [20]:
to_summarize

,Unnamed: 0,Statement,Summary 1,Contradicting statement,Summary 2,GPT (original statements),GPT (summaries)
0,0,Die AfD will ein investitions- und innovations...,Die AfD setzt sich für ein wirtschaftliches Um...,Die AfD befürwortet einen gesetzlichen Mindest...,Die AfD unterstützt die Einführung eines geset...,No,Inconsistency.\n\nReasoning: Text 1 suggests t...
1,1,Wir wollen Deutschland reformieren. Das geht n...,Summary: The text advocates for a reform of Ge...,Die AfD will die derzeit zur Erhebung ausgeset...,Die AfD plädiert für die Abschaffung der Vermö...,No,Text 1 and Text 2 both discuss changes to the ...
2,2,Wir wollen die staatliche Macht über den Bürge...,"Wir streben an, die staatliche Macht über den ...",Kinder unter drei Jahren fühlen sich am wohlst...,"Die AfD argumentiert, dass Kinder unter drei J...",Yes,Consistent.\n\nBoth texts describe policies pr...
3,3,Wir wollen die staatliche Macht über den Bürge...,Die AfD will die staatliche Kontrolle und den ...,Den Menschen in unserem Land wieder Sicherheit...,The AfD aims to ensure security for workers an...,Yes,Inconsistency.\n\nReasoning: Text 1 states tha...
4,4,Beatrix von Storch: Die AfD verurteilt Antisem...,"Summary: Beatrix von Storch, deputy spokeswoma...","Wir Deutschen, also unser Volk, sind das einzi...",Summary: The text expresses a critical view to...,Yes,Unrelated.\n\nThe first text discusses the iss...
5,5,Georg Pazderski: Keinen Millimeter für Antisem...,Summary: Georg Pazderski criticizes the existi...,"Wir Deutschen, also unser Volk, sind das einzi...",Summary: The speaker criticizes the focus on G...,Yes,Answer: Unrelated.\n\nReasoning: Text 1 focuse...
6,6,Die AfD will den Einfluss der Parteien auf das...,Die AfD möchte den politischen Einfluss auf da...,"Museen, Orchester und Theater sind in der Pfli...","Museums, orchestras, and theaters in Saxony-An...",No,Consistent.\n\nReasoning: Text 1 describes a p...
7,7,Die AfD will den Einfluss der Parteien auf das...,"Die AfD setzt sich dafür ein, den politischen ...",Heute jährt sich die Befreiung von Auschwitz-B...,Summary: Today marks the anniversary of the li...,No,Consistent.\n\nReasoning:\nText 1 discusses th...
8,8,NaN,NaN,NS-Verbrechen dürfen niemals in Vergessenheit ...,"Beatrix von Storch, Vice Chair and Antisemitis...",No,NaN
9,9,Die AfD bekennt sich uneingeschränkt zur Glaub...,"The AfD fully supports freedom of belief, cons...",NaN,NaN,Yes,NaN


In [12]:
to_summarize['Summary 1'][0]

"Die AfD setzt sich für ein wirtschaftliches Umfeld ein, das Investitionen und Innovationen fördert, indem sie weitreichende Deregulierung und Bürokratieabbau fordert. Die Partei betrachtet die aktuelle Menge an Regulierungen als zu umfangreich und ineffizient und möchte den unternehmerischen Geist durch die Beseitigung bürokratischer Hindernisse neu beleben. Der Ansatz beinhaltet nicht nur eine Verbesserung der Regelgestaltung, sondern auch eine Überprüfung der Notwendigkeit bestehender Regelungen.\n\nTopics: ['economy']\nStances: {'deregulation': 'positive', 'bureaucracy': 'negative'}"

In [6]:
assistant = client.beta.assistants.create(
  name="Summarization Assistant",
  instructions='''
  Your task is to summarize the given text. The summary should contain important aspects of the original text. The main goal is that to reflect claims or stances expressed in the text.
  
  Example:
  Input: The AfD wants to reduce the influence of the parties on cultural life, strengthen non-profit private cultural foundations and civic cultural initiatives and generally align cultural policy with professional quality criteria and economic reason rather than with political opportunities. 
  The current narrowing of the German culture of remembrance to the time of National Socialism must be broken down in favor of an expanded view of history that also includes the positive, identity-forming aspects of German history.
  
  Summary: We believe that Germany should get rid of the Rememberance culture and promote a more positive portrayal of its history.
  
  Topics: ['culture']
  Stances: {'rememberance_culture': 'negative', 'culture_intervention': 'negative'}
''',
  model="gpt-4-turbo",
)

In [7]:
assistant.id

'asst_2eeGKT8rHwL2UYXLbVjiZGez'

In [14]:
for i in range(len(to_summarize['Contradicting statement'])):
    statement = to_summarize['Contradicting statement'][i]
    if str(statement) not in ["NaN", "nan"]:
        try:
            # Create a thread with a message.
            thread = client.beta.threads.create(
                messages=[
                    {
                        "role": "user",
                        # Update this with the query you want to use.
                        "content": statement,
                    }
                ]
            )

            # Submit the thread to the assistant (as a new run).
            run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant.id)

            # Wait for run to complete.
            while run.status != "completed":
                run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
                time.sleep(1)
            else:
                print(f"🏁 Run {i} Completed!")

            # Get the latest message from the thread.
            message_response = client.beta.threads.messages.list(thread_id=thread.id)
            messages = message_response.data

            # Print the latest message.
            latest_message = messages[0]
            
            to_summarize['Summary 2'][i] = latest_message.content[0].text.value
        except:
            print(f"Run {i} went wrong")

        to_summarize.to_csv("summarized_by_gpt.csv", index=False)

🏁 Run 0 Completed!


C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_265932\3252815569.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_summarize['Summary 2'][i] = latest_message.content[0].text.value
C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_265932\3252815569.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Die AfD unterstützt die Einführung eines gesetzlichen Mindestlohns, um die schwachen Positionen von Niedriglohnempfängern gegenüber Arbeitgebern zu stärken und Armut sowie Abhängigkeit von staatlichen Sozialleistungen zu verhindern. Die Partei sieht prekäre Arbeitsverhältnisse als schädlich für den gesellschaftlichen Wohlstand an und schlägt vor, den Anteil von Beschäftigten in Leih- oder Werkverträgen auf 15 Prozent zu begrenzen. 

🏁 Run 1 Completed!
🏁 Run 2 Completed!
🏁 Run 3 Completed!
🏁 Run 4 Completed!
🏁 Run 5 Completed!
🏁 Run 6 Completed!
🏁 Run 7 Completed!
🏁 Run 8 Completed!
🏁 Run 11 Completed!
🏁 Run 12 Completed!
🏁 Run 13 Completed!
🏁 Run 14 Completed!
🏁 Run 15 Completed!
🏁 Run 16 Completed!
🏁 Run 17 Completed!
🏁 Run 18 Completed!
🏁 Run 19 Completed!
🏁 Run 20 Completed!
🏁 Run 21 Completed!
🏁 Run 22 Completed!
🏁 Run 23 Completed!
🏁 Run 24 Completed!
🏁 Run 25 Completed!
🏁 Run 26 Completed!
🏁 Run 27 Completed!
🏁 Run 28 Completed!
🏁 Run 29 Completed!
🏁 Run 30 Completed!
🏁 Run 31 Completed!
🏁 Run 32 Completed!
🏁 Run 33 Completed!
🏁 Run 34 Completed!
🏁 Run 35 Completed!
🏁 Run 36 Completed!


In [17]:
assistant = client.beta.assistants.create(
  name="Contradictions between summaries",
  instructions='''
    Your task is to take two input texts and tell whether there is an inconsistency between them or not. If there is an inconsistency, briefly explain where it is. If there are no inconsistencies, just answer "Consistent". If the texts are unrelated, just say "Unrelated". 

    Example 1.
    Text 1: We are against bureaucracy and want to decrease regulations. We want to make the rules easier for the companies.
    Text 2: We care about the working class and support increasing the minimum wage. 
    Reasoning: Making rules easier for companies, as Text 1 suggests, might imply more freedom in specifying the conditions in the contracts and setting the minimum hourly rate, as well as the minimum wage, which is inconsistent with Text 2. Therefore, the answer is: Inconsisent.
    Answer: Inconsisent.
''',
  model="gpt-4-turbo",
)

In [19]:
for i in range(len(to_summarize)):
    text1 = to_summarize['Summary 1'][i]
    text2 = to_summarize['Summary 2'][i]
    if str(text1) not in ["NaN", "nan"] and str(text2) not in ["NaN", "nan"]:
        try:
            # Create a thread with a message.
            thread = client.beta.threads.create(
                messages=[
                    {
                        "role": "user",
                        # Update this with the query you want to use.
                        "content": f"Text 1: {text1}. Text 2: {text2}",
                    }
                ]
            )

            # Submit the thread to the assistant (as a new run).
            run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant.id)

            # Wait for run to complete.
            while run.status != "completed":
                run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
                time.sleep(1)
            else:
                print(f"🏁 Run {i} Completed!")

            # Get the latest message from the thread.
            message_response = client.beta.threads.messages.list(thread_id=thread.id)
            messages = message_response.data

            # Print the latest message.
            latest_message = messages[0]
            
            to_summarize['GPT (summaries)'][i] = latest_message.content[0].text.value
        except:
            print(f"Run {i} went wrong")

        to_summarize.to_csv("summarized_by_gpt.csv", index=False)

🏁 Run 0 Completed!


C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_265932\3689488211.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_summarize['GPT (summaries)'][i] = latest_message.content[0].text.value
C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_265932\3689488211.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Inconsistency.

Reasoning: Text 1 suggests that the AfD promotes a reduction in regulations and bureaucracy, aiming to revive the entrepreneurial spirit by eliminating bureaucratic obstacles. They view existing regulations as overly elaborate and inefficient. On the other hand, Text 2 indicates that the AfD supports the establishment of a statutory minimum wage and stricter regulations on temporary employment contracts, which

🏁 Run 1 Completed!
🏁 Run 2 Completed!
🏁 Run 3 Completed!
🏁 Run 4 Completed!
🏁 Run 5 Completed!
🏁 Run 6 Completed!
🏁 Run 7 Completed!
🏁 Run 11 Completed!
🏁 Run 12 Completed!
🏁 Run 13 Completed!
🏁 Run 14 Completed!
🏁 Run 15 Completed!
🏁 Run 16 Completed!
🏁 Run 17 Completed!
🏁 Run 18 Completed!
🏁 Run 19 Completed!
🏁 Run 20 Completed!
🏁 Run 21 Completed!
🏁 Run 22 Completed!
🏁 Run 23 Completed!
🏁 Run 24 Completed!
🏁 Run 25 Completed!
🏁 Run 26 Completed!
🏁 Run 27 Completed!
🏁 Run 28 Completed!
🏁 Run 29 Completed!
🏁 Run 30 Completed!
🏁 Run 31 Completed!
🏁 Run 32 Completed!
🏁 Run 33 Completed!
🏁 Run 34 Completed!
🏁 Run 35 Completed!
🏁 Run 36 Completed!
